In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import torch

import matplotlib.pyplot as plt

from torch.utils.data import Dataset
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam

from scipy.stats import spearmanr

from sklearn.metrics import mean_squared_error
from sklearn import metrics

import os
from tqdm import tqdm

In [2]:
#load training data (will be put in a function later)  
path = os.getcwd()
for i in range(3) :

    path = os.path.dirname(path)

path += '/data/'
train_df = pd.read_csv(path + 'train_v1.csv',index_col="seq_id")
train_df = train_df.drop(columns=['data_source'])
train_df = train_df.dropna()
train_df.head()


# train_df = pd.read_csv('train_v1copy.csv' ,index_col="seq_id")
# train_df = train_df.drop(['data_source'], axis=1)
# len(train_df)

,protein_sequence,pH,tm
seq_id,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5


In [3]:
train_df['length'] = train_df['protein_sequence'].str.len()
train_df['length'].value_counts()
train_df['length'].nunique()
#print(train_df)
train_df_without_tm = train_df.copy()
del train_df_without_tm['tm']
print(train_df_without_tm)

                                         protein_sequence   pH  length
seq_id                                                                
0       AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...  7.0     341
1       AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...  7.0     286
2       AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...  7.0     497
3       AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...  7.0     265
4       AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...  7.0    1451
...                                                   ...  ...     ...
31385   YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...  7.0     549
31386   YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...  7.0     469
31387   YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...  7.0     128
31388   YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...  7.0     593
31389   YYVPDEYWQSLEVAHKLTFGYGYLTWEWVQGIRSYVYPLLIAGLYK...  7.0     537

[28695 rows x 3 columns]


In [4]:
def split_data(df,tm,ratio):
    num_row = len(df)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[:index_split]
    index_te = indices[index_split:]

    x_tr = df.iloc[index_tr]
    x_te = df.iloc[index_te]
    y_tr = tm.iloc[index_tr]
    y_te = tm.iloc[index_te]
    
    return y_tr, x_tr, y_te, x_te


In [5]:
def Scoring(df_te, df_predicted):
    df = {
    "true": df_te['tm'],
    "predicted": df_predicted['tm']
}
    pearson = df.corr(method='pearson')
    rmse = mean_squared_error(df_te['tm'], df_predicted['tm'], squared=False)
    auc = metrics.roc_auc_score(df_te['tm'], df_predicted['tm'])
    
    print('Pearson: %.3f, RMSE %.3f, AUC: %.3f' %(pearson, rmse, auc))
    return pearson, rmse, auc

In [6]:
y_tr, x_tr, y_te, x_te = split_data(train_df_without_tm, train_df['tm'],0.8)

In [10]:
!nvidia-smi

Mon Dec 12 11:39:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   39C    P8    12W / 235W |   7875MiB /  7982MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
!python --version

Python 3.9.13


In [12]:
!nvcc -V && which nvcc

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
/usr/bin/nvcc


In [24]:
!conda install -c rapidsai -c nvidia -c conda-forge -c defaults rapidsai::rapids python=3.7 cudatoolkit=10.1

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ ^C
- 

In [34]:
import sys
!cp ../rapids.0.13.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/


cp: cannot stat '../rapids.0.13.0': No such file or directory
/bin/bash: line 0: cd: /opt/conda/envs/: No such file or directory
cp: cannot stat '/opt/conda/envs/rapids/lib/libxgboost.so': No such file or directory


In [35]:
import cudf

#if torch.cuda.is_available():  
#print('RAPIDS version',cuda.__version__)

print('Train shape:', train_df.shape )
train_df.head()

ModuleNotFoundError: No module named 'cudf'

In [9]:
vc = train_df.length.value_counts()
print(vc)


164     748
231     318
455     245
155     243
148     241
       ... 
1366      1
1690      1
2069      1
1872      1
1345      1
Name: length, Length: 1962, dtype: int64


In [10]:
#find minimum number operations to convert str1 to str2 (FROM GEEKSFORGEEKS)
# 
def edit_distance(str1, str2):
	m = len(str1)
	n = len(str2)

	# Create a table to store results of subproblems
	dp = [[0 for x in range(n + 1)] for x in range(m + 1)]

	# Fill d[][] in bottom up manner
	for i in range(m + 1):
		for j in range(n + 1):

			# If first string is empty, only option is to
			# insert all characters of second string
			if i == 0:
				dp[i][j] = j    # Min. operations = j

			# If second string is empty, only option is to
			# remove all characters of second string
			elif j == 0:
				dp[i][j] = i    # Min. operations = i

			# If last characters are same, ignore last char
			# and recur for remaining string
			elif str1[i-1] == str2[j-1]:
				dp[i][j] = dp[i-1][j-1]
 
            # If last character are different, consider all
            # possibilities and find minimum
			else:
				dp[i][j] = 1 + min(dp[i][j-1],        # Insert,
								dp[i-1][j],        # Remove
								dp[i-1][j-1])    # Replace
 
	return dp[m][n]
 
 

# This code is contributed by Bhavya Jain


print(len(train_df.protein_sequence[65]), len(train_df.protein_sequence[128]))

# str1 = train_df.protein_sequence[578]
# str2 = train_df.protein_sequence[579]	

str1 = train_df.protein_sequence[65]
str2 = train_df.protein_sequence[128]

edit_distance(str1, str2)




56 57


47

In [11]:
def edit_distance_matrix(df):
    vlign = []
    for id in df.index:
        
        hlign = []
        for jd in df.index:
            hlign.append(edit_distance(df[id], df[jd]))
        vlign.append(hlign)
        
    sums = vlign.copy()
    sums = np.sum(sums, axis=1)
    wildtype = np.argmin(sums)
    return vlign 

 #edit_distance_matrix(tmp[:50]) 

In [ ]:
#import cuDF
train_df['group'] = -1
train_df['type'] = 'unknown'

grp = 0
wildtype_ids = []

# MUTATION THRESHOLD
M_THRESHOLD = 10
# INSERTION DELETION THRESHOLD
D_THRESHOLD = 3

for k in tqdm(range(0, len(vc))): 
    c = vc.index[k]
    # SUBSET OF TRAIN DATA WITH SAME PROTEIN LENGTH PLUS MINUS D_THRESHOLD
    tmp = train_df.loc[(train_df.length>=c-D_THRESHOLD)&(train_df.length<=c+D_THRESHOLD)&(train_df.group==-1)]
    if len(tmp)<=1: break
    
    # COMPUTE LEVENSTEIN DISTANCE
    #x = tmp.protein_sequence.str.edit_distance_matrix() #original but cudf is needed
    x = edit_distance_matrix(tmp['protein_sequence']) #fonction fait maison 
    x = np.array( pd.Series(x).values.tolist() )
    #wildtype_ids.append(wt)
    #train_df.loc[wt,'type'] = 'WT'
    
    # COUNT HOW MANY MUTATIONS WE SEE
    mutation = []
    for kk in range(1,M_THRESHOLD+1):
        mutation.append( len( np.unique( np.where( x==kk )[0] ) ) )
        
    # FIND RELATED ROWS IN TRAIN WITH M_THRESHOLD MUTATIONS OR LESS
    y = np.unique( np.where( (x>0)&(x<=M_THRESHOLD) )[0] )
    seen = []
    for j in y:
        if j in seen: continue
        i = np.where( np.array(x[j,])<=M_THRESHOLD )[0]
        seen += list(i)
        idx = tmp.iloc[i,].index 
        train_df.loc[idx,'group'] = grp
        grp += 1
    ct = vc.iloc[k]
    ct2 = len(tmp)
    print(f'k={k} len={c} ct={ct} ct2={ct2} dist_ct={mutation}') 
    
      

  0%|                                                                                                                                                                                                                 | 0/1962 [00:00<?, ?it/s]

In [ ]:
#deleting proteins without groups
train_df_groups = train_df[train_df.group!= -1]

In [ ]:
# saving train_df in CSV file 
train_df_more_columns= train_df_groups.copy()
train_df_more_columns.to_csv('train_df_more_columns.csv') 


In [ ]:
# display code
for k in range(10):
    print('#'*25)
    print(f'### GROUP {k}')
    print('#'*25)
    display( train_df.loc[train_df.group==k] )

In [ ]:
# find minimum number operations to convert str1 to str2

 
def editDistDP(str1, str2, m, n):
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)]
 
    # Fill d[][] in bottom up manner
    for i in range(m + 1):
        for j in range(n + 1):
 
            # If first string is empty, only option is to
            # insert all characters of second string
            if i == 0:
                dp[i][j] = j    # Min. operations = j
 
            # If second string is empty, only option is to
            # remove all characters of second string
            elif j == 0:
                dp[i][j] = i    # Min. operations = i
 
            # If last characters are same, ignore last char
            # and recur for remaining string
            elif str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
 
            # If last character are different, consider all
            # possibilities and find minimum
            else:
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert
                                   dp[i-1][j],        # Remove
                                   dp[i-1][j-1])    # Replace
 
    return dp[m][n]
 
 
# Driver code
str1 = train_df['protein_sequence'][0]
str2 = train_df['protein_sequence'][1]
 
print(editDistDP(str1, str2, len(str1), len(str2)))
# This code is contributed by Bhavya Jain

In [ ]:
nrow = train_df.loc[train_df.group==-1].shape[0] 
mx = train_df.group.max() 
train_df.loc[train_df.group==-1,'group'] = np.arange(nrow) + mx + 1.